In [ ]:
from pykalman import KalmanFilter
import numpy as np
import matplotlib.pyplot as plt
import time
from rdp import rdp
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 10]

In [ ]:
import gpxpy

with open('m8t.gpx') as fh:
    gpx_file = gpxpy.parse(fh)

In [ ]:
segment = gpx_file.tracks[0].segments[1]

import pandas as pd
coords = pd.DataFrame([{'idx': i,
                        'lat': p.latitude, 
                        'lon': p.longitude, 
                        'ele': p.elevation,
                        'speed': p.speed,
                        'time': p.time} for i, p in enumerate(segment.points)])
coords.set_index('time', inplace=True)
coords.head()

In [ ]:
measurements_all = np.ma.masked_invalid(coords[['lon', 'lat', 'speed']].values)
times_all = range(measurements_all.shape[0])

In [ ]:
#coords_part = coords[['lon', 'lat', 'speed']][750:880].values
#coords_part = coords[['lon', 'lat', 'speed']][750:980].values
coords_part = coords[['lon', 'lat', 'speed']][750:830].values
measurements_part = np.ma.masked_invalid(coords_part)
times_part = range(measurements_part.shape[0])

#lon_plt = plt.subplot(311)
#lon_plt.plot(times_part, measurements_part[:, 0], 'b-')
#lat_plt = plt.subplot(312)
#lat_plt.plot(times_part, measurements_part[:, 1], 'b-')
#speed_plt = plt.subplot(313)
#speed_plt.plot(times_part, measurements_part[:, 2], 'b-')

#data_plt = plt.subplot(324)
#data_plt.plot(measurements_part[:, 0], measurements_part[:, 1], 'b-')


In [ ]:
class KF:
    def __init__(self, q, r, f = 1, h = 1):
        self.X0 = 0.0 # predicted state
        self.P0 = 0.0 # predicted covariance

        self.F = 0.0 # factor of real value to previous real value
        self.Q = 0.0 # measurement noise
        self.H = 0.0 # factor of measured value to real value
        self.R = 0.0 # environment noise

        self.Q = q
        self.R = r
        self.F = f
        self.H = h
    def set(self, state, covariance):
        self.State = state
        self.Covariance = covariance
    def correct(self, data):
        # time update - prediction
        self.X0 = self.F * self.State
        self.P0 = self.F * self.Covariance * self.F + self.Q

        # measurement update - correction
        K = self.H * self.P0 / (self.H * self.P0 * self.H + self.R)
        self.State = self.X0 + K * (data - self.H * self.X0)
        self.Covariance = (1 - K * self.H) * self.P0

def smooth(arr):
    out = []
    kf = KF(q=2, r=15, f=1, h=1)
    kf.set(arr[0], 0.1)
    for d in arr:
        kf.correct(d)
        out.append(kf.State)
    return out

In [ ]:
# Fixed description of vars
# Renamed vars in correct()
class KF:
    def __init__(self, q, r, f = 1, h = 1):
        self.F = 0.0 # factor of real value to previous real value
        self.Q = 0.0 # covariance of untracked influences
        self.H = 0.0 # factor of measured value to real value
        self.R = 0.0 # covariance of measurement noise
        self.Xprev = 0.0 # previous state
        self.Pprev = 0.0 # previous covariance

        self.Q = q
        self.R = r
        self.F = f
        self.H = h
    def set(self, x, covariance):
        self.Xprev = x
        self.Pprev = covariance
    def correct(self, data):
        # time update - prediction
        Xpredicted = self.F * self.Xprev
        Ppredicted = self.F * self.Pprev * self.F + self.Q

        # measurement update - correction
        K = self.H * Ppredicted / (self.H * Ppredicted * self.H + self.R)
        
        Xnew = Xpredicted + K * (data - self.H * Xpredicted)
        Pnew = Ppredicted - K * self.H * Ppredicted
        
        self.Xprev = Xnew
        self.Pprev = Pnew

def smooth(arr):
    out = []
    #kf = KF(q=2, r=15, f=1, h=1)
    kf = KF(q=0.2, r=0.1, f=1, h=1)
    kf.set(arr[0], 0.0)
    for d in arr:
        kf.correct(d)
        out.append(kf.Xprev)
    return out

In [ ]:
def smoothAndDraw(arr, arr_plt):
    out = smooth(arr)
    arr_plt.plot(
        times_part, arr, 'b-',
        times_part, out, 'g-',
    )
    return out
    
lon_plt = plt.subplot(311)
new_lon = smoothAndDraw(measurements_part[:, 0], lon_plt)
lat_plt = plt.subplot(312)
new_lat = smoothAndDraw(measurements_part[:, 1], lat_plt)
speed_plt = plt.subplot(313)
x = smoothAndDraw(measurements_part[:, 2], speed_plt)




In [ ]:
plt.plot(
    measurements_part[:, 0], measurements_part[:, 1], 'b-',
    new_lon, new_lat, 'g-'
)
